In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize
from sklearn.datasets import make_regression

In [4]:
import matplotlib.pyplot as plt

In [45]:
data = make_regression(n_samples=1, n_features=1, n_informative=4, n_targets=1, noise=4.0, coef=True)

In [46]:
data

(array([[0.33622623]]), array(27.50974343), array(77.05396328))

In [47]:
data[1]

array(27.50974343)

In [48]:
data[0]

array([[0.33622623]])

In [50]:
np.insert(data[0], 1, data[1], axis=1)

array([[ 0.33622623, 27.50974343]])

In [210]:
def create_dataset_helper(num_samples,low,high,n,d,num_updates):
    #assumes n = 1 for original dataset
    #probably move to keras/tf backend as n grows
    #flatten array for now but maybe keep structure (similar to cnns)
    original_ds = np.random.randint(low,high+1,size=(num_samples,n,d+1))
    updates = np.random.randint(low,high+1,size=(num_samples,num_updates,d+1))
    betas = np.zeros((num_samples,num_updates+1,d))
    lm = LinearRegression(fit_intercept=False)
    for i in range(num_samples):
        if i % 1000 == 0:
            print(i)
        ds = original_ds[i]
        lm.fit(ds[:,:-1], ds[:,-1])
        betas[i,0] = lm.coef_
        for j in range(num_updates):
            ds = np.append(ds,[updates[i,j]],axis=0) #careful with pointers
            lm.fit(ds[:,:-1],ds[:,-1])
            betas[i,j+1] = lm.coef_
            
    original_ds_reshape = np.reshape(original_ds, (num_samples, n*(d+1)))
    updates_reshape = np.reshape(updates, (num_samples, num_updates*(d+1)))
    X = np.append(updates_reshape, betas.reshape(num_samples,d*(num_updates+1)),axis=1)
    normalize(X, copy=False)
    y = original_ds_reshape
    return X,y

In [207]:
def create_dataset(num_samples=1000,low=-100,high=100,n=1,d=1,num_updates=1):
    Xtrain,ytrain = create_dataset_helper(num_samples,low,high,n,d,num_updates)
    Xtest,ytest = create_dataset_helper((int)(.2*num_samples), low,high,n,d,num_updates)
    dataset = {"Xtrain": Xtrain, "ytrain": ytrain,
               "Xtest": Xtest, "ytest": ytest}
    for data in dataset:
        df = pd.DataFrame(dataset[data])
        df.to_csv(data+".csv",index=None,header=None)
    return dataset

In [208]:
create_dataset(100,n=1,d=2,num_updates=3)

0
0


{'Xtrain': array([[-7.20000000e+01, -9.00000000e+01,  2.70000000e+01, ...,
         -8.46589085e-01,  6.67009907e-01, -7.87354240e-01],
        [-1.80000000e+01, -3.90000000e+01, -7.00000000e+01, ...,
          1.24221794e+00,  1.22499944e+00,  1.00115303e+00],
        [ 7.40000000e+01,  5.10000000e+01,  8.80000000e+01, ...,
         -6.58518055e-01,  1.69388105e+00, -5.27432333e-01],
        ...,
        [-3.60000000e+01, -5.00000000e+00,  3.00000000e+01, ...,
         -3.45061443e-01,  7.29915056e-02,  1.44694178e-01],
        [-7.90000000e+01, -2.90000000e+01,  5.00000000e+00, ...,
          5.78395987e-01, -2.25950777e-01,  8.05333422e-01],
        [ 5.90000000e+01, -5.50000000e+01,  7.00000000e+00, ...,
          4.16275172e-01, -7.81419716e-02, -1.63728677e-01]]),
 'ytrain': array([[ 58, -31,  67],
        [  3, -54, -32],
        [-14, -49,  -8],
        [-11,  14,  -1],
        [ 27,  96,  76],
        [ 20,  72,  66],
        [ 84,  51,  53],
        [ -4, -72, -12],
        [